In [75]:
# this notebook implements logical xor function with backprop
import numpy as np
import tensorflow as tf

X = np.array([[1.0,0.0,0.0,1.0],[0.0,0.0,1.0,1.0]],dtype=np.float32) # 2x4 matrix
Y = np.array([[1.0,0.0,1.0,0.0]],dtype=np.float32)                   # 1x4 vector
print("X: ", X)
print("Y: ", Y)

# parameters of neural net
W1 = tf.Variable(tf.random_uniform([2,2],minval=-1,maxval=1,seed=1)) # 2x2 matrix
b1 = tf.Variable([[0.0],[0.0]])                                      # 2x1 vector
W2 = tf.Variable(tf.random_uniform([1,2],minval=-1,maxval=1,seed=1)) # 1x2 vector
b2 = tf.Variable(0.0)                                                # scalar

# computational graph for forward pass
Z1 = tf.matmul(W1,X)   # 2x4 matrix
Z2 = Z1 + b1           # 2x4 matrix
Z3 = tf.nn.sigmoid(Z2) # 2x4 matrix
Z4 = tf.matmul(W2,Z3)  # 1x4 vector
Z5 = Z4 + b2           # 1x4 vector
Yp = tf.nn.sigmoid(Z5) # 1x4 vector

# computational graph for backward pass
dYp = Yp - Y                               # 1x4 vector
dZ5 = Yp*(1.0 - Yp)*dYp                    # 1x4 vector
dZ4 = dZ5                                  # 1x4 vector
dZ3 = tf.matmul(tf.transpose(W2),dZ4)      # 2x4 matrix
dZ2 = Z3 * (1.0 - Z3) * dZ3                # 2x4 matrix
dZ1 = dZ2                                  # 2x4 matrix
dW2 = tf.matmul(dZ4, tf.transpose(Z3))     # 1x2 vector
db2 = tf.reduce_sum(dZ5)                   # scalar
dW1 = tf.matmul(dZ1,tf.transpose(X))       # 2x2 matrix
db1 = tf.reduce_sum(dZ2,1,keep_dims=True)  # 2x1 vector

X:  [[ 1.  0.  0.  1.]
 [ 0.  0.  1.  1.]]
Y:  [[ 1.  0.  1.  0.]]


In [76]:
# init node for variable initialization
init = tf.global_variables_initializer()

# square loss function
l2loss = 0.5*tf.reduce_mean(tf.square(Yp-Y))

learning_rate = 8.0

# training operators
training_op1a = tf.assign(W1,W1 - learning_rate * dW1 )
training_op1b = tf.assign(b1,b1 - learning_rate * db1 )
training_op2a = tf.assign(W2,W2 - learning_rate * dW2 )
training_op2b = tf.assign(b2,b2 - learning_rate * db2 )

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(1000):
        # gradient descent to adjust parameters
        sess.run([training_op1a,training_op1b,training_op2a,training_op2b])
        
        if epoch%100 == 0:
            error = l2loss.eval()
            print(error)
    out = Yp.eval()

print("Learned output: ",out)
print("Ideal output: ",Y)

0.125274
0.00501187
0.000747451
0.00039308
0.000264586
0.000198704
0.000158782
0.000132058
0.000112942
9.86036e-05
Learned output:  [[ 0.98470771  0.01327089  0.98768425  0.01178161]]
Ideal output:  [[ 1.  0.  1.  0.]]
